In [113]:
import pandas as pd
import datetime as dt
import json

In [102]:
path = '../network_read/hist_data/2022-04-16/AQU/4'
path_parts = path.split('/')
date = path_parts[-3]
track_id = path_parts[-2]
race_number = path_parts[-1]
race_dt = dt.datetime.strptime(date, '%Y-%m-%d')

pd.set_option('display.max_columns', None)

In [103]:
details = pd.read_csv(f'{path}/details.csv')

In [104]:
bis = pd.read_csv(f'{path}/bis.csv')

In [105]:
# Remove scratched
bis = bis[bis['scratched'] == False]

check_for_nans = ['powerRating','daysOff','horseWins','horseStarts','avgClassRating','highSpeed','avgSpeed','lastClassRating',
                  'avgDistance','numRaces','early','middle','finish','starts','wins','places','shows','finishPosition',
                  ]
for col in check_for_nans:
    if bis[col].isna().sum() != 0:
        # (TODO) return (skip this race)
        pass

bis.loc[:,'age'] = race_dt.year - bis.loc[:,'birthday']
bis.loc[:,'winPayoff'] = bis.loc[:,'winPayoff'] / bis.loc[:,'betAmount']
bis.loc[:,'placePayoff'] = bis.loc[:,'placePayoff'] / bis.loc[:,'betAmount']
bis.loc[:,'showPayoff'] = bis.loc[:,'showPayoff'] / bis.loc[:,'betAmount']

# for horseName, jockey, trainer, owner, sire, damSire, dam. keep them for now. later, find which ones are most common and one-hot on those.

to_keep = [#'runnerId',
           'horseName','jockey','trainer','owner','weight','sire','damSire','dam','age','sex','powerRating',
           'daysOff','horseWins','horseStarts','avgClassRating','highSpeed','avgSpeed','lastClassRating','avgDistance',
           'finishPosition']
bis = bis[to_keep]

In [106]:
bis

,horseName,jockey,trainer,owner,weight,sire,damSire,dam,age,sex,powerRating,daysOff,horseWins,horseStarts,avgClassRating,highSpeed,avgSpeed,lastClassRating,avgDistance,finishPosition
0,Alms,Castellano Javier,Stidham Michael,"Godolphin, LLC",118,City Zip,Empire Maker,Charity Belle,5,M,101.1,41,2,8,105.0,105.0,96.0,106.0,97.0,4.0
2,Plum Ali,Franco Manuel,Clement Christophe,"Dubb, Michael, Madaket Stables LLC and Caruso,...",120,First Samurai,Stroll,Skipping,4,F,107.3,152,4,11,110.0,111.0,101.0,111.0,98.0,1.0
4,Flower Point,Lezcano Jose,Mcgaughey Iii Claude R,"Feiger, Steve",120,Point Of Entry,Lewis Michael,Fab Flowers,6,M,103.6,140,1,4,101.0,104.0,96.0,104.0,100.0,3.0
5,Technical Analysis (Ire),Davis Dylan,Brown Chad C,"Klaravich Stables, Inc.",123,Kingman (Gb),Sea The Stars (Ire),Sealife (Ire),4,F,106.3,181,4,7,95.0,106.0,95.0,113.0,93.0,2.0


In [107]:
dist_to_meters = {'f': 201.168,
                  'mtr': 1,     # mtr (meter) comes before m (mile) in search
                  'm': 1609.34,
                  'y': 0.9144}
details.loc[0,'distance'] = details.loc[0,'distance'] * dist_to_meters[details.loc[0,'distance_unit'].lower()]

In [108]:
details = details[['distance','surface_name','race_type_code','race_class']]

In [115]:
bis_list = bis.to_dict('records')
for idx, bi in enumerate(bis_list):
    details.loc[0,f'bi{idx}'] = json.dumps(bi)


In [116]:
details

,distance,surface_name,race_type_code,race_class,bi0,bi1,bi2,bi3
0,1609.344,Turf,T,Stakes,"{""horseName"": ""Alms"", ""jockey"": ""Castellano Ja...","{""horseName"": ""Plum Ali"", ""jockey"": ""Franco Ma...","{""horseName"": ""Flower Point"", ""jockey"": ""Lezca...","{""horseName"": ""Technical Analysis (Ire)"", ""joc..."


## data_hist

In [120]:
import numpy as np
import pandas as pd
import os
import itertools
import matplotlib.pyplot as plt
import json
from collections import defaultdict

In [162]:
horseracing_path = f'./'

data = pd.read_csv(horseracing_path + 'hist_data.csv', header=None)
num_horses = 10

In [165]:
def extract_dict(dict_str):
    try:
        return json.loads(dict_str)
    except:
        return None
def get_uniques(df, bi_feature, threshold=1):
    uniques = defaultdict(lambda: 0)
    for r in range(df.shape[0]):
        for c in range(4,df.shape[1]):
            d = extract_dict(df.iloc[r,c])
            if d == None:
                continue
            uniques[d[bi_feature]] += 1
    threshold_uniques = {k:v for k,v in uniques.items() if v >= threshold}
    return {k: v for k, v in sorted(threshold_uniques.items(), key=lambda x: -x[1])}

In [167]:
get_uniques(data, 'horseName', 10)

{'Intervention': 13,
 'Abnaa': 12,
 'Coase': 12,
 'Soyounique (Ire)': 10,
 'Out For Justice': 10,
 'Beverly Park': 10}

In [169]:
get_uniques(data, 'jockey', 300)

{'Gaffalione Tyler': 412,
 'Saez Luis': 394,
 'Hernandez Harry': 355,
 'Gomez Jose A': 345,
 'Herrera Diego A': 332,
 'Ortiz, Jr. Irad': 324,
 'Leon Sonny': 312,
 'Ortiz Jose L': 312,
 'Alvarado Junior': 312,
 'Camacho Samy': 305,
 'Davis Dylan': 301,
 'Mccarthy Trevor': 301}

In [172]:
get_uniques(data, 'owner', 100)

{'Bruno Schickedanz': 161,
 'Hollywood Syndicate (Nom: Mr A Delpech)': 135,
 'Edward C. Allred': 129,
 'End Zone Athletics, Inc.': 121,
 'End Zone Athletics Inc. (Karl Broberg and Matt Johanson)': 119,
 'Steve F. Williams': 116,
 'Mark N. Hibdon': 103,
 'Danny R. Caldwell': 101}

In [175]:
get_uniques(data, 'dam', 15)

{'Culotte': 18,
 'Celestial Being': 18,
 'Ready Reply': 16,
 'Katie Katella': 16,
 'Along Came Casey (Ire)': 16,
 "Jimmy's Jewel": 15,
 "Sara's Smile": 15,
 'Guanahani': 15,
 'Cabo Sunrise': 15,
 'Arealhotlover': 15,
 'April Eyes': 15,
 'Restless Summer': 15,
 'Miss Silky Cat': 15,
 'Royal Grand Slam': 15}

In [178]:
get_uniques(data, 'damSire', 500)

{nan: 2635,
 "Unbridled's Song": 686,
 'Malibu Moon': 624,
 "Giant's Causeway": 620,
 'Smart Strike': 581,
 'Tapit': 575,
 'Tiznow': 567,
 'Distorted Humor': 541,
 "Medaglia D'oro": 502}

In [181]:
get_uniques(data, 'sire', 300)

{'Into Mischief': 442,
 'Munnings': 388,
 'Goldencents': 361,
 'Hard Spun': 339,
 'Cairo Prince': 336,
 'Twirling Candy': 336,
 'Tapiture': 326,
 'Street Sense': 317,
 'Congrats': 316,
 'Violence': 309,
 'Exaggerator': 302,
 'Creative Cause': 302,
 'Quality Road': 302}

In [183]:
get_uniques(data, 'trainer', 200)

{'Asmussen Steven M': 640,
 'Broberg Karl': 436,
 'Wong Jonathan': 418,
 'Ness Jamie': 276,
 'Maker Michael J': 257,
 'Diodoro Robertino': 235,
 'Farrior Anthony': 227,
 'Casse Mark E': 224,
 'P. A. Peter, South Africa': 208,
 'Pletcher Todd A': 206,
 'Joseph, Jr. Saffie A': 206,
 'Cox Brad H': 205}